In [916]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [917]:
# DATA INGESTION

# Fetch the data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [918]:
# Check the DataFrame
train_df.tail(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


In [919]:
# PREPROCESSING

# Create a fake Survived column for test data
test_df.loc[:,"Survived"] = -1

In [920]:
# Concatenate both training and test data
data = pd.concat([train_df,test_df]).reset_index(drop=True)

In [921]:
# FEATURE ENGINEERING

# Drop non interested columns
data.drop(['PassengerId','Name','Cabin','Ticket'], axis=1, inplace=True)
data.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [922]:
# Impute missing values in the entire 'Age' & 'Fare' columns using median
data['Age']=data['Age'].fillna(data['Age'].median(skipna=True)) #skipna=True to exclude NaN values when computing the result
data['Fare']=data['Fare'].fillna(data['Fare'].mean(skipna=True))

In [923]:
# Fill in missing values with the most common embarkation value - 'S' * Parch
data["Embarked"].fillna(data['Embarked'].value_counts().idxmax(), inplace=True)
data["Parch"].fillna({'Parch': data['Parch'].value_counts().idxmax()}, inplace=True)

/var/folders/xg/dpkd6v6s38sd016lvk5lz21w0000gn/T/ipykernel_3495/2855294128.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Embarked"].fillna(data['Embarked'].value_counts().idxmax(), inplace=True)
/var/folders/xg/dpkd6v6s38sd016lvk5lz21w0000gn/T/ipykernel_3495/2855294128.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whi

In [924]:
data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [925]:
# Cap the Fare at a reasonable threshold, e.g., 300
data['Fare'] = data['Fare'].clip(upper=300)

In [926]:
# Bin Age groups
bins = [0, 3, 12, 18, 35, 55, 65, float('inf')]
labels = ['Infant/Toddler', 'Child', 'Teenager/Adolescent', 'Young Adult', 'Middle-Aged Adult', 'Older Adult', 'Senior/Elderly']
# Create a new column 'AgeGroup' with the binned age categories
data['AgeGroup'] = pd.cut(data['Age'], bins=bins, labels=labels, include_lowest=True)

In [927]:
data.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeGroup
0,0,3,male,22.0,1,0,7.2500,S,Young Adult
1,1,1,female,38.0,1,0,71.2833,C,Middle-Aged Adult
2,1,3,female,26.0,0,0,7.9250,S,Young Adult
3,1,1,female,35.0,1,0,53.1000,S,Young Adult
4,0,3,male,35.0,0,0,8.0500,S,Young Adult
5,0,3,male,28.0,0,0,8.4583,Q,Young Adult
6,0,1,male,54.0,0,0,51.8625,S,Middle-Aged Adult
7,0,3,male,2.0,3,1,21.0750,S,Infant/Toddler
8,1,3,female,27.0,0,2,11.1333,S,Young Adult
9,1,2,female,14.0,1,0,30.0708,C,Teenager/Adolescent


In [928]:
# Bin SibSp and Parch to group the data 
#data['SibSp'] = pd.cut(data['SibSp'], bins=[-1, 0, 1, 3, 8], labels=['0', '1', '2', '3'])
#data['Parch'] = pd.cut(data['Parch'], bins=[-1, 0, 1, 3, 6], labels=['0', '1', '2', '3'])

In [929]:
# Join two features to simplify modelling
data['TravelAlone']=np.where((data["SibSp"]+data["Parch"])>0, 0, 1)

In [930]:

# Drop the columns
data.drop(['Age','SibSp','Parch'], axis=1, inplace=True)

In [931]:

# Convert categorical variables or the ones that represent such ('Pclass') to dummy/numerical variables (one-hot encoding)
data = pd.get_dummies(data, columns=['Embarked', 'Sex', 'AgeGroup','Pclass'], dtype=int, drop_first=True) # drop_first to avoid collinearity

In [932]:
# Now handle the male outliers (> 63 years) by filling them with the median age
#data.loc[(data['Sex_male']) & (data['Age'] > 63), 'Age'] = data['Age'].median()

In [933]:
data.head(10)

,Survived,Fare,TravelAlone,Embarked_Q,Embarked_S,Sex_male,AgeGroup_Child,AgeGroup_Teenager/Adolescent,AgeGroup_Young Adult,AgeGroup_Middle-Aged Adult,AgeGroup_Older Adult,AgeGroup_Senior/Elderly,Pclass_2,Pclass_3
0,0,7.2500,0,0,1,1,0,0,1,0,0,0,0,1
1,1,71.2833,0,0,0,0,0,0,0,1,0,0,0,0
2,1,7.9250,1,0,1,0,0,0,1,0,0,0,0,1
3,1,53.1000,0,0,1,0,0,0,1,0,0,0,0,0
4,0,8.0500,1,0,1,1,0,0,1,0,0,0,0,1
5,0,8.4583,1,1,0,1,0,0,1,0,0,0,0,1
6,0,51.8625,1,0,1,1,0,0,0,1,0,0,0,0
7,0,21.0750,0,0,1,1,0,0,0,0,0,0,0,1
8,1,11.1333,0,0,1,0,0,0,1,0,0,0,0,1
9,1,30.0708,0,0,0,0,0,1,0,0,0,0,1,0


In [934]:
# split the training and test data again
train_df = data[data.Survived != -1].reset_index(drop=True) 
test_df = data[data.Survived == -1].reset_index(drop=True)

In [935]:
# CROSS-VALIDATION
# We create a new column called kfold and fill it with -1
train_df['kfold'] = -1

# The next step is to randomize the rows of the data
train_df = train_df.sample(frac=1,random_state=32).reset_index(drop=True)

# Fetch the targets
y = train_df.Survived.values

# Inititate the kfold class
kf = model_selection.StratifiedKFold(n_splits=5)

# fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=train_df,y=y)):
    train_df.loc[v_,'kfold'] = f

In [936]:
train_df.head()

,Survived,Fare,TravelAlone,Embarked_Q,Embarked_S,Sex_male,AgeGroup_Child,AgeGroup_Teenager/Adolescent,AgeGroup_Young Adult,AgeGroup_Middle-Aged Adult,AgeGroup_Older Adult,AgeGroup_Senior/Elderly,Pclass_2,Pclass_3,kfold
0,0,8.0500,1,0,1,1,0,0,1,0,0,0,0,1,0
1,1,133.6500,0,0,1,0,0,0,1,0,0,0,0,0,0
2,1,89.1042,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,7.7375,1,1,0,1,0,0,1,0,0,0,0,1,0
4,0,0.0000,1,0,1,1,0,0,1,0,0,0,1,0,0


In [937]:
# Collect accuracies
lst = []

# Loop folds
for fold in range(0,5):
    # Training data is where kfold is not equal to provided fold
    df_train = train_df[train_df.kfold != fold].reset_index(drop=True)
    
    # Validation data is where kfold is equal to provided fold
    df_valid = train_df[train_df.kfold == fold].reset_index(drop=True)

    # Drop the Survived and kfold column from dataframe and convert it to a numpy array
    x_train = df_train.drop(['Survived','kfold'],axis=1).values
    y_train = df_train.Survived.values

    # Similarly, for validation
    x_valid = df_valid.drop(['Survived','kfold'],axis=1).values
    y_valid = df_valid.Survived.values

    scaler = StandardScaler() # Solved ConvergenceWarning
    x_train = scaler.fit_transform(x_train)
    x_valid = scaler.transform(x_valid)

    # INITIALIZE THE MODEL & FINE-TUNING
    model = LogisticRegression(random_state=32)

    # Fit the model on training data
    model.fit(x_train,y_train)

    # Create predictions for validations samples
    preds = model.predict(x_valid)

    # Calculate & print accuracy
    accuracy = metrics.accuracy_score(y_valid,preds)
    print(f"Fold = {fold}, Accuracy = {accuracy}")

    lst.append(accuracy)

Fold = 0, Accuracy = 0.7988826815642458
Fold = 1, Accuracy = 0.8202247191011236
Fold = 2, Accuracy = 0.8033707865168539
Fold = 3, Accuracy = 0.7921348314606742
Fold = 4, Accuracy = 0.7865168539325843


In [938]:
Average = sum(lst) / len(lst) 
print(f"Average accuracy = {Average}")

Average accuracy = 0.8002259745150964


In [939]:
# Make predictions on the test data
test_predictions = model.predict(test_df.drop('Survived',axis=1).values)

In [940]:
# Prepare the submission file
submission = pd.read_csv('../data/submission.csv')
submission['Survived'] = test_predictions
submission.tail(60)

,PassengerId,Survived
358,1250,0
359,1251,0
360,1252,0
361,1253,0
362,1254,0
363,1255,0
364,1256,0
365,1257,0
366,1258,0
367,1259,0


In [941]:
# Save to CSV
submission.to_csv('../data/submission.csv', index=False)
print("Submission file saved as 'submission.csv'.")

Submission file saved as 'submission.csv'.
